In [2]:
import numpy as np
import pandas as pd
import pickle

In [77]:
import ortools
from ortools.linear_solver import pywraplp
from ortools.graph import pywrapgraph

In [4]:
df = pd.read_pickle("train.pkl")
vectors = df["Vector"]

In [5]:
df

,Text,Author,Vector
0,U.S. Senators on Tuesday sharply criticized a ...,AaronPressman,"[-0.59266317, 3.8505063, -8.919514, 1.9474734,..."
1,Two members of Congress criticised the Federal...,AaronPressman,"[-2.0250783, -0.4225978, -3.5096471, -2.018811..."
2,Commuters stuck in traffic on the Leesburg Pik...,AaronPressman,"[5.966192, -1.5661882, -2.1684153, -5.9777327,..."
3,A broad coalition of corporations went to Capi...,AaronPressman,"[5.257664, -5.1360354, -4.637482, -3.0475514, ..."
4,"On the Internet, where new products come and g...",AaronPressman,"[4.779245, -3.3181891, -6.280878, -3.8667245, ..."
...,...,...,...
2495,China has scored new successes in its fight ag...,WilliamKazer,"[-0.20546861, 4.163672, -1.6106012, -3.1293087..."
2496,China has scored new successes in its fight ag...,WilliamKazer,"[-0.20965889, 4.847113, -1.7382896, -3.3654442..."
2497,China is on target with plans to to promote 10...,WilliamKazer,"[-2.3209264, 3.1882226, 1.1266638, -3.8210783,..."
2498,China may need to adjust the mix of its treasu...,WilliamKazer,"[-6.7202153, 7.621765, -3.2023606, -0.9481356,..."


In [6]:
import ast 

distances = np.zeros((2500,2500))
from scipy.spatial import distance

for i in range(len(vectors)):
    a = vectors[i]
    for j in range(len(vectors)):
        b = vectors[j]
        dst = distance.euclidean(a, b)
        distances[i,j] = dst

In [9]:
#pickle.dump(distances, open("distances.pkl", "wb"))
distances = pd.read_pickle("distances.pkl")

In [11]:
C = []
authors = df["Author"].unique()

for i in authors:
    df_sub = df[df["Author"] == i]
    C.append(df_sub.index.values)

In [33]:
C;

In [14]:
a = 0.5
a_1 = np.ceil(1/a)
colors = 50
k = 25
lam = 15
n = 2500

In [7]:
x = []
y = []
a = 0.5
a_1 = np.ceil(1/a)
colors = 50
k = 25
lam = 15
n = 2500
F_small = []

solver = pywraplp.Solver('LP',
                         pywraplp.Solver.GLOP_LINEAR_PROGRAMMING)

objective = solver.Objective()
for i in range(n):
    x.append([])
    y.append(solver.NumVar(0.0, 1.0,'y'))
    for j in range(n):
        x[i].append(solver.NumVar(0.0, 1.0,'x'))
        objective.SetCoefficient(x[i][j], 1)
objective.SetMaximization()

constraint = []
for j in range(n):
    constraint.append(solver.Constraint(1, 1))
    for i in range(n):
        constraint[j].SetCoefficient(x[i][j],1)

for i in range(n):
    for j in range(n):
        constraint.append(solver.Constraint(-solver.infinity(), 0))
        constraint[-1].SetCoefficient(x[i][j],1)
        constraint[-1].SetCoefficient(y[i],-1)

for i in range(colors):
    constraint.append(solver.Constraint(-solver.infinity(), 0))
    for j in range(n):
        constraint[-1].SetCoefficient(x[i][j],-a)
    if j in C[i]:
        constraint[-1].SetCoefficient(x[i][j],1-a)

for i in range(n):
    constraint.append(solver.Constraint(0, solver.infinity()))
    for j in range(n):
        constraint[-1].SetCoefficient(x[i][j],1)
        constraint[-1].SetCoefficient(y[i],-a_1)

constraint.append(solver.Constraint(0, k))
for i in range(n):
    constraint[-1].SetCoefficient(y[i],1)

for i in range(n):
    for j in range(n):
        if distances[i,j] > lam:
            constraint.append(solver.Constraint(0, 0))
            constraint[-1].SetCoefficient(x[i][j],1)

status = solver.Solve()
status == solver.OPTIMAL

True

In [9]:
X_sol = np.zeros((n,n))
Y_sol = []
for i in range(n): 
    Y_sol.append(y[i].solution_value())
    for j in range(n):
        X_sol[i,j] = x[i][j].solution_value()

In [18]:
#pickle.dump(X_sol, open("X15.pkl", "wb"))
#pickle.dump(Y_sol, open("Y15.pkl", "wb"))

In [229]:
x_sol = pd.read_pickle("X15.pkl")
y_sol = pd.read_pickle("Y15.pkl")

In [284]:
z = 0
v = 0
for i in range(n):
    z += y_sol[i]
    for j in range(n):
        v+= x_sol[i,j]
print(v,z)

2500.0000000002515 24.999999999999865


In [231]:
F = []
for i in range(n):
    if y_sol[i]>0:
        F.append(i)

In [25]:
def greedy_lambda(F, F_distances, lamb):
    removed = []
    Fcopy1 = F.copy()
    for i in F:
        if i not in removed:
            for j in F:
                if j not in removed:
                    if (i == j):
                        pass
                    elif (F_distances[i,j] < 2*lamb):
                        #print(F_distances[i,j])
                        Fcopy1.remove(j)
                        removed.append(j)
    return Fcopy1

In [232]:
len(F)

335

In [233]:
dist = np.ones((n,n))*100000
for i in F:
    for j in F:
        if i != j:
            dist[i,j] = distances[i,j]

In [234]:
F_prime = set()
for i in F:
        val = 0
        for j in F_prime.copy():
            if dist[i,j] < 16:
                val = 1
        if val ==0:
            F_prime.add(i)

In [235]:
F_prime = greedy_lambda(F,distances,10)

In [236]:
dist1 = np.ones((n,n))*100000
for i in F:
    for j in F_prime:
        if i != j:
            dist1[i,j] = distances[i,j]

In [258]:
d = {}
h = 0
for f in F:
    if f in F_prime: 
        d[f] = []
        d[f].append(f)
    else:
        for i in F_prime:
            #if dist1[f,i] == min(dist1[f]) and dist1[f,i] < 20:
            if dist1[f,i] < 20:
                if i in d:
                    d[i].append(f)
                else:
                    d[i] = []
                    d[i].append(f)
                break

In [259]:
x_new = np.zeros((n,n))
for j in np.arange(n):
    for i in F_prime:
        for element in d[i]:
            x_new[i,j] += x_sol[element,j]

In [260]:
dict_enum = {}
num = 2500
for i in F_prime:
    for c in np.arange(len(C)):
        dict_enum[(i,c)] = num
        num += 1

ind = len(F_prime)* 50 + 2500
dict_enum['s'] = ind
dict_enum['t'] = ind+1

In [78]:
dict_enum;

In [261]:
start_nodes = []
end_nodes = []
capacity = []

for j in range(n):
    start_nodes.append(int(ind))
    end_nodes.append(int(j))
    capacity.append(int(1))

arcindex = n
dict_arcs = {}

for color in range(len(C)):
    for index in C[color]:
        for i in F_prime:
            if x_new[i,index] > 0:
                start_nodes.append(int(index))
                dictind = dict_enum[(i,color)]
                end_nodes.append(int(dictind))
                capacity.append(int(1))
                dict_arcs[arcindex] = (index,(i,color))
                arcindex += 1
                
for i in F_prime:
    for color in range(len(C)):
        val = 0
        for index in C[color]:
            val += x_new[i,index]
        val = np.ceil(val)
        dictind = dict_enum[(i,color)]
        start_nodes.append(int(dictind))
        end_nodes.append(int(i))
        capacity.append(int(val))

for i in F_prime:
    val = 0
    for j in range(n):
        val += x_new[i,j]
    val = np.ceil(val)
    start_nodes.append(int(i))
    end_nodes.append(int(ind+1))
    capacity.append(int(val))


max_flow = pywrapgraph.SimpleMaxFlow()
for index in range(0, len(start_nodes)):
    max_flow.AddArcWithCapacity(start_nodes[index], end_nodes[index], capacity[index])

    
new_x1 = np.zeros((n,n))
if max_flow.Solve(ind, ind+1) == max_flow.OPTIMAL:
    for index in np.arange(n,arcindex):
        if max_flow.Flow(int(index)) > 0:
            j = dict_arcs[index][0]
            i = dict_arcs[index][1][0]
            col = dict_arcs[index][1][1]
            new_x1[i,j] = 1

In [262]:
eeee = 0
suma = {}
for i in F_prime:
    suma[i] = 0
    for j in range(n):
        suma[i]+= new_x1[i,j]
suma

{5: 634.0,
 8: 350.0,
 51: 413.0,
 115: 20.0,
 172: 540.0,
 300: 117.0,
 357: 254.0,
 407: 115.0,
 727: 10.0,
 973: 22.0,
 1294: 8.0,
 1671: 6.0}

In [285]:
listforc = {}
seen = []
for i in range(n):
    if sum(new_x1[i]) > 0:
        listforc[i] = []
        for j in range(n):
            if new_x1[i,j] > 0:
                listforc[i].append(j)
                seen.append(j)
#listforc

In [286]:
for i in range(n):
    if i not in seen:
        #if i not in F_prime:
            #print(i)

SyntaxError: unexpected EOF while parsing (<ipython-input-286-a1eb3e5162d5>, line 4)

In [287]:
tot = 0
for i in range(n):
    for j in range(n):
        tot += new_x1[i,j]
tot

2489.0

In [288]:
dists = []
for i in listforc:
    for j in listforc[i]:
        dists.append(distances[i,j])
max(dists)

31.37601089477539

In [297]:
Fassign = {}

for i in listforc:
    Fassign[i] = {}
    for index in listforc[i]:
        for color in range(len(C)):
            Fassign[i][color] = 0
            if index in C[color]:
                Fassign[i][color] += 1
 